In [1]:
import requests
import pandas as pd
import numpy as np
import json
from json import loads

In [2]:
counter = 0

In [3]:
#Query all data from a year
#Add counter for the number of extractions
data=[]
for i in range(1,6):
    counter+=1
    url = "https://api-football-v1.p.rapidapi.com/v3/players"

    querystring = {"league":"39","season":"2002","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
   }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    if response.text == "'{'message': 'You have exceeded the rate limit per minute for your plan, BASIC, by the API provider'}'":
        print('Restart next loop at:', i)
        break
    else: 
        data.append(response.text)
else: 
    print('Start next loop at 30')
#Good but have to pop last pages which were over my quota per minute

Start next loop at 30


In [4]:
counter

5

In [5]:
#complete year array
for i in range(30,33):
    counter+=1
    url = "https://api-football-v1.p.rapidapi.com/v3/players"
    
    #YEAR TO CHANGE IN QUERY
    querystring = {"league":"39","season":"2003","page":str(i)}

    headers = {
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com",
        "X-RapidAPI-Key": "d356c39fe7mshf3ec31c0312f244p17df85jsn2bc9fd933f3f"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    data.append(response.text)

In [6]:
tableau=[]
for i in range(len(data)):
    tableau.append(i)
#Format data to json
jsonData=list(map(lambda i: loads(data[i]),tableau))
len(jsonData)

8

In [7]:
len(jsonData[4]['response'])

20

In [8]:
#Create prototype array which will be column names in the df
proto= ["id"\
, "firstname"\
, "lastname"\
, "age"\
, "nationality"\
, "height"\
, "weight"\
, "injured"\
, "teamID"\
, "teamname"\
, "appearences"\
, "lineups"\
, "minutes"\
, "position"\
, "rating"\
, "captain"\
, "subIn"\
, "subOut"\
, "bench"\
, "totalShots"\
, "shotsOn"\
, "totalGoals"\
, "goalsConceded"\
, "goalAssists"\
, "goalSaves"\
, "totalPasses"\
, "keyPasses"\
, "passAccuracy"\
, "totalTackles"\
, "blocks"\
, "interceptions"\
, "duelsTotal"\
, "duelsWon"\
, "dribbleAttempts"\
, "dribblesSuccess"\
, "dribblesPast"\
, "foulsDrawn"\
, "foulsCommitted"\
, "cardsYel"\
, "cardsYelRed"\
, "cardsRed"\
, "penaltyWon"\
, "penaltyCommited"\
, "penaltyScored"\
, "penaltyMissed"\
, "penaltySaved"]

In [9]:
#Verify the final page items to adjust creation of the df
pagesNum = len(jsonData)
finalPageItems = len(jsonData[pagesNum-1]['response'])
finalPageItems

0

In [10]:
finalArr = []
#Keys to iterate through
playerKeys = ['id', 'firstname', 'lastname', 'age', 'nationality', 'height', 'weight', 'injured']
#Create a dict to optimize code
stat = {
    'team':['id','name'],
    'games':['appearences','lineups','minutes','position','rating','captain'],
    'substitutes':['in','out','bench'],
    'shots':['total','on'],
    'goals':['total','conceded','assists','saves'],
    'passes':['total','key','accuracy'],
    'tackles':['total','blocks','interceptions'],
    'duels':['total','won'],
    'dribbles':['attempts','success','past'],
    'fouls':['drawn','committed'],
    'cards':['yellow','yellowred','red'],
    'penalty':['won','commited','scored','missed','saved']
}

iterable = 0
pagesNum = len(jsonData)
finalPageItems = len(jsonData[pagesNum-1]['response'])
for i in range(pagesNum):
    
    #To adjust for the last page
    if i == pagesNum-1: 
        iterable = finalPageItems
    else:
        iterable = 20
        
    for j in range(iterable):
        to_fill = []
        
        for pkey in playerKeys:
            to_fill.append(jsonData[i]['response'][j]['player'][pkey])
        for key in stat:
            for innerkey in stat[key]:
                to_fill.append(jsonData[i]['response'][j]['statistics'][0][key][innerkey])        
        finalArr.append(to_fill)

IndexError: list index out of range

In [11]:
len(finalArr)

100

In [12]:
#Vector representing one data point
arr = np.array(finalArr)

In [13]:
#create the df
df = pd.DataFrame(arr, columns = proto)
df

,id,firstname,lastname,age,nationality,height,weight,injured,teamID,teamname,...,foulsDrawn,foulsCommitted,cardsYel,cardsYelRed,cardsRed,penaltyWon,penaltyCommited,penaltyScored,penaltyMissed,penaltySaved
0,18241,Mark,Yeates,36,Republic of Ireland,175 cm,66 kg,False,47,Tottenham,...,None,None,0,0,0,None,None,None,None,None
1,19153,Gareth,Barry,39,England,183 cm,78 kg,False,66,Aston Villa,...,None,None,0,0,0,None,None,None,None,None
2,19203,Ashley,Cole,41,England,176 cm,66 kg,False,42,Arsenal,...,None,None,0,0,0,None,None,None,None,None
3,19417,Liam,Ridgewell,37,England,188 cm,77 kg,False,66,Aston Villa,...,None,None,0,0,0,None,None,None,None,None
4,50978,Justin,Hoyte,36,Trinidad and Tobago,180 cm,68 kg,False,42,Arsenal,...,None,None,0,0,0,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,118155,Kevin Alistair,Kyle,40,Scotland,193 cm,83 kg,False,746,Sunderland,...,None,None,0,0,0,None,None,None,None,None
96,118905,Darren,Purse,44,England,188 cm,79 kg,False,54,Birmingham,...,None,None,0,0,0,None,None,None,None,None
97,124184,Roy,Carroll,44,Northern Ireland,188 cm,80 kg,False,33,Manchester United,...,None,None,0,0,0,None,None,None,None,None
98,155103,Jon,Macken,45,Republic of Ireland,178 cm,78 kg,False,50,Manchester City,...,None,None,0,0,0,None,None,None,None,None


In [14]:
#Cell kept to check if data is good
gb = df.groupby(df['lastname']).count()
df[df['teamname']=='Tottenham']

,id,firstname,lastname,age,nationality,height,weight,injured,teamID,teamname,...,foulsDrawn,foulsCommitted,cardsYel,cardsYelRed,cardsRed,penaltyWon,penaltyCommited,penaltyScored,penaltyMissed,penaltySaved
0,18241,Mark,Yeates,36,Republic of Ireland,175 cm,66 kg,False,47,Tottenham,...,None,None,0,0,0,None,None,None,None,None
53,231007,Mauricio,Taricco,48,Argentina,175 cm,73 kg,False,47,Tottenham,...,None,None,1,0,0,None,None,None,None,None
79,233875,Robert,Burch,38,England,188 cm,82 kg,False,47,Tottenham,...,None,None,0,0,0,None,None,None,None,None


In [15]:
df['season'] = '2002/2003'

In [174]:
#Export the df to a csv
df.to_csv('data2002&2003.csv')

In [175]:
counter

20